# **Diplomová práce - výpočty**
### **Jméno:** Alois Primes
### **Název práce:** MODULARNÍ HORIZONTÁLNÍ KOTEL – HRSG
### **Zadání**
**Stručná charakteristika problematiky úkolu:**

Práce je zaměřena na návrh horizontálního kotle využívajícího spaliny za plynovou spalovací turbínou pro produkci páry k dalšímu energetickému využití. Kotel by měl být moderní konstrukce, zahrnující
maximální prefabrikaci transportních bloků (modulů), která je ovlivněna podmínkami dopravního profilu na místo stavby. Kotel bude řešen jako modulární, s horizontálním prouděním spalin, konvekčním
výparníkem, výhřevnými plochami ohříváku vody (ekonomizéry) a přehříváky páry s regulací teploty páry vstřikem napájecí vody, odebírané mezi dvěma ekonomizéry.

**Základní parametry kotle jsou:** parní výkon 76 t/h, tlak 3,1 MPa, výstupní teplota 370 °C. Teplota
napájecí vody kotle je 71°C. Další parametry kotle a odpadních spalin budou upřesněny v rámci
realizace práce.

**Cíle diplomové práce:**

Zpracovat tepelný výpočet kotle.
Zpracovat rozměrový návrh kotle, včetně základní výkresové dokumentace kotle a dispozičního
řešení.
Vyčíslit dosaženou účinnost kotle při vztažné teplotě 25 °C, použijte přímou metodu.
Zpracovat diagram průběhu teploty pracovního média a spalin a stanovit bilanci vstřiku.

In [1]:
# Instalace a aktualizace modulů

# !pip install --upgrade pip
# !pip install --upgrade tabulate
# !pip install --upgrade forallpeople
# !pip install --upgrade handcalcs
# !pip install --upgrade pytexit
# !pip install --upgrade pyXSteam
# !pip install --upgrade CoolProp

In [2]:
# Import knihoven

import pandas as pd
from CoolProp.CoolProp import PropsSI
import handcalcs.render
from math import pi, e
from pyXSteam.XSteam import XSteam
from tabulate import tabulate

In [3]:
# Nastavení prostředí a modulů

%decimal_separator ,
steam = XSteam(XSteam.UNIT_SYSTEM_MKS) # Jednotky: m/kg/sec/°C/bar/W

## Definice pomocných funkcí

In [4]:
def density(prvek, teplota, tlak):

	"""
	Zadej požadavek ve formátu density('prvek', teplota [K], tlak [BAR])
	Příklad rho_CO2 = density('CO2', 293.15, 5.5)
	Vypočte hustotu CO2 pro teplotu 20 °C (293.15 K) a 5.5 baru (550 000 Pa)
    JEDNOTKA: kg/m^3
	"""

	rho = PropsSI('D', 'T', teplota, 'P', tlak*100000, prvek)
	return rho

In [5]:
def conductivity(prvek, teplota, tlak):

	"""
	Zadej požadavek ve formátu conductivity('prvek', teplota [K], tlak [bar])
	Příklad lambda_CO2 = conductivity('CO2', 293.15, 1.01325)
	Vypočte vodivost CO2 pro teplotu 20 °C (293.15 K) a 1.01325 baru (101 325 Pa)
    JEDNOTKA: W/m*K
	"""

	lambda_prvek = PropsSI('conductivity', 'T', teplota, 'P', tlak*100000, prvek)
	return lambda_prvek

In [6]:
def enthalpy(prvek, teplota, tlak):

	"""
	Zadej požadavek ve formátu enthalpy('prvek', teplota [K], tlak [bar])
	Příklad h_CO2 = enthalpy('CO2', 293.15, 1.01325)
	Vypočte entalpii CO2 pro teplotu 20 °C (293.15 K) a 1.01325 baru (101 325 Pa)
	JEDNOTKA: J/kg
	"""

	h_prvek = PropsSI('H', 'T', teplota, 'P', tlak*100000, prvek)
	return h_prvek

In [7]:
def viscosity(prvek, teplota, tlak):

	"""
	Zadej požadavek ve formátu viscosity('prvek', teplota [K], tlak [bar])
	Příklad eta_CO2 = viscosity('CO2', 293.15, 1.01325)
	Vypočte viskozitu CO2 pro teplotu 20 °C (293.15 K) a 1.01325 baru (101 325 Pa)
	JEDNOTKA: Pa.s
	"""

	eta_prvek = PropsSI('viscosity', 'T', teplota, 'P', tlak*100000, prvek)
	return eta_prvek

In [8]:
def cp(prvek, teplota, tlak):

	"""
	Zadej požadavek ve formátu cp('prvek', teplota [K], tlak [bar])
	Příklad cp_CO2 = cp('CO2', 293.15, 1.01325)
	Vypočte měrnou tepelnou kapacitu CO2 pro teplotu 20 °C (293.15 K) a 1.01325 baru (101 325 Pa)
	JEDNOTKA: J/kg*K
	"""

	cp_prvek = PropsSI('C', 'T', teplota, 'P', tlak*100000, prvek)
	return cp_prvek

In [9]:
def kin_viscosity(prvek, teplota, tlak):

	"""
	Zadej požadavek ve formátu kin_viscosity('prvek', teplota [K], tlak [bar])
	Příklad nu_CO2 = kin_viscosity('CO2', 293.15, 1.01325)
	Vypočte kinematickou viskozitu CO2 pro teplotu 20 °C (293.15 K) a 1.01325 baru (101 325 Pa)
	JEDNOTKA: m^2/s
	"""
	eta_prvek = PropsSI('viscosity', 'T', teplota, 'P', tlak*100000, prvek)
	rho_prvek = PropsSI('D', 'T', teplota, 'P', tlak*100000, prvek)
	nu_prvek = eta_prvek/rho_prvek
	return nu_prvek

## Základní parametry kotle

In [10]:
%%render params 2
m_p = 76        #t/h
p_out = 3.1     #MPa
t_out = 370     #°C
t_nv = 71       #°C
m_vst = 0.36    #kg/s
t_vst = 217     #°C

<IPython.core.display.Latex object>

## Parametry spalin ze spalovací turbíny

In [11]:
R = 8.314462618 #J/kg/mol
t_ref = 100+273.15 #°C

#### koeficienty polynomů pro výpočet entalpií spalin

In [12]:
# N2
a_1_N2 = 0.3694063e+01
a_2_N2 = -0.1334139e-02
a_3_N2 = 0.2652067e-05
a_4_N2 = -0.9775312e-09
a_5_N2 = -0.9983837e-13
a_6_N2 = -0.9758881e+03

# O2
a_1_O2 = 0.3721461e+01
a_2_O2 = -0.2518398e-02
a_3_O2 = 0.8589429e-05
a_4_O2 = -0.8305377e-08
a_5_O2 = 0.2710013e-11
a_6_O2 = -0.9701834e+03

# CO2
a_1_CO2 = 0.2171539e+01
a_2_CO2 = 0.1038499e-01
a_3_CO2 = -0.1074106e-04
a_4_CO2 = 0.6350127e-08
a_5_CO2 = -0.1629149e-11
a_6_CO2 = -0.9159472e+03

In [13]:
%%render params 3
m_sp = 130.1    #kg/s
t_sp = 562.0    #°C

# Objemové složení spalin
x_N2 = 0.7457
x_O2 = 0.1289
x_H2O = 0.0801
x_CO2 = 0.0364
x_Ar = 0.0089

<IPython.core.display.Latex object>

In [14]:
%%render params 3

# Molární hmotnost složek spalin (z podkladů)
M_N2 = 28.0134e-03      #kg/kmol
M_O2 = 31.9988e-03      #kg/kmol
M_H2O = 18.0152e-03     #kg/kmol
M_CO2 = 44.0095e-03     #kg/kmol
M_Ar = 39.9480e-03      #kg/kmol

<IPython.core.display.Latex object>

In [15]:
%%render params

# Hustota složek spalin (z podkladů)
rho_N2 = 1.2504     #kg/m3
rho_O2 = 1.4290     #kg/m3
rho_H2O = 0.8038    #kg/m3
rho_CO2 = 1.9772    #kg/m3
rho_Ar = 1.7840     #kg/m3
rho_sp = x_N2*rho_N2 + x_O2*rho_O2 + x_H2O*rho_H2O + x_CO2*rho_CO2 + x_Ar*rho_Ar #kg/m3

<IPython.core.display.Latex object>

In [16]:
%%render params 3

# Měrný objem složek spalin (z podkladů)
v_N2 = 22.403     #m3/kg
v_O2 = 22.392     #m3/kg
v_H2O = 22.414    #m3/kg
v_CO2 = 22.258    #m3/kg
v_Ar = 22.393     #m3/kg

<IPython.core.display.Latex object>

In [38]:
%%render params
i_N2 = (a_1_N2 + a_2_N2*t_ref/2 + a_3_N2*t_ref**2/3 + a_4_N2*t_ref**3/4 + a_5_N2*t_ref**4/5 + a_6_N2/t_ref)*((R*t_ref)/M_N2)/1000 #kJ/m3n
i_O2 = (a_1_O2 + a_2_O2*t_ref/2 + a_3_O2*t_ref**2/3 + a_4_O2*t_ref**3/4 + a_5_O2*t_ref**4/5 + a_6_O2/t_ref)*((R*t_ref)/M_O2)/1000 #kJ/m3n
i_CO2 = (a_1_CO2 + a_2_CO2*t_ref/2 + a_3_CO2*t_ref**2/3 + a_4_CO2*t_ref**3/4 + a_5_CO2*t_ref**4/5 + a_6_CO2/t_ref)*((R*t_ref)/M_CO2)/1000 #kJ/m3n
i_Ar = enthalpy('Ar', t_ref, 1.01)/1000 #blbě
i_H2O = enthalpy('H2O',t_ref, 1.01)/1000 #blbě

i_sp = x_N2*i_N2 + x_O2*i_O2 + x_CO2*i_CO2 + x_H2O*i_H2O + x_Ar*i_Ar

<IPython.core.display.Latex object>

#### Bod 1 (Konec druhého přehříváku, výstupní pára)

In [18]:
%%render params
t_1 = t_out                     #°C
p_1 = p_out                     #MPa
i_1 = steam.h_pt(p_1*10,t_1)    #kJ/kg
v_1 = steam.v_pt(p_1*10,t_1)    #m3/kg
x_1 = steam.x_ph(p_1*10,i_1)    #-

<IPython.core.display.Latex object>

#### Bod 3 (Konec prvního přehříváku, před vstřikem)

In [19]:
%%render params
t_3 = 348                       #°C
p_3 = 3.33                      #MPa
i_3 = steam.h_pt(p_3*10,t_3)    #kJ/kg
v_3 = steam.v_pt(p_3*10,t_3)    #m3/kg
x_3 = steam.x_ph(p_3*10,i_3)    #-

<IPython.core.display.Latex object>

#### Bod 4 (Začátek prvního přehříváku, po druhém výparníku)

In [20]:
%%render params
p_4 = 3.38                      #MPa
t_4 = steam.tsat_p(p_4*10)      #°C
i_4 = steam.hV_p(p_4*10)        #kJ/kg
v_4 = steam.vV_p(p_4*10)        #m3/kg
x_4 = steam.x_ph(p_4*10,i_4)    #-

<IPython.core.display.Latex object>

#### Bod 5 (začátek prvního výparníku, kde je sytá kapalina)

In [21]:
%%render params
p_5 = 3.38                      #MPa
t_5 = steam.tsat_p(p_5*10)      #°C
i_5 = steam.hL_p(p_5*10)        #kJ/kg
v_5 = steam.vL_p(p_5*10)        #m3/kg
x_5 = steam.x_ph(p_5*10,i_5)    #-

<IPython.core.display.Latex object>

#### Bod 6 (Za posledním ekonomizérem, před vstupem do výparníku)

In [22]:
%%render params
p_6 = 3.68                      #MPa
t_6 = 236                       #°C
i_6 = steam.h_pt(p_6*10, t_6)   #kJ/kg
v_6 = steam.v_pt(p_6*10, t_6)   #m3/kg
x_6 = steam.x_ph(p_6*10,i_6)    #-

<IPython.core.display.Latex object>

#### Bod 7 (mezi ekonomizéry)

In [23]:
%%render params
p_7 = 3.74                      #MPa
t_7 = 217                       #°C
i_7 = steam.h_pt(p_7*10, t_7)   #kJ/kg
v_7 = steam.v_pt(p_7*10, t_7)   #m3/kg
x_7 = steam.x_ph(p_7*10,i_7)    #-


<IPython.core.display.Latex object>

#### Bod 2 (Začátek druhého přehříváku, po vstřiku)

In [24]:
%%render params
p_2 = p_3                           #MPa
i_2 = (20.74*i_3 + 0.36*i_7)/21.10  #kJ/kg
t_2 = steam.t_ph(p_2*10, i_2)       #°C
v_2 = steam.v_ph(p_2*10, i_2)       #m3/kg
x_2 = steam.x_ph(p_2*10,i_2)        #-

<IPython.core.display.Latex object>

#### Bod 8 (Před začátkem ekonomizéru, vstup napájecí vody)

In [25]:
%%render params
t_8 = 71                         #°C
p_8 = 4                          #MPa (NEVIM)
i_8 = steam.h_pt(p_8*10,t_8)     #kJ/kg
v_8 = steam.v_pt(p_8*10,t_8)     #m3/kg
x_8 = steam.x_ph(p_8*10,i_8)    #-

<IPython.core.display.Latex object>

### Tabulka bodová pro vodu a páru

In [26]:
bodova_tabulka = pd.DataFrame(
    {
        "T [°C]": (t_1, t_2, t_3, t_4, t_5, t_6, t_7, t_8),
        "p [MPa]": (p_1, p_2, p_3, p_4, p_5, p_6, p_7, p_8),
        "i [kJ/kg]": (i_1, i_2, i_3, i_4, i_5, i_6, i_7, i_8),
        "v [m3/kg]": (v_1, v_2, v_3, v_4, v_5, v_6, v_7, v_8),
        "x [-]": (x_1, x_2, x_3, x_4, x_5, x_6, x_7, x_8),
    },
    index = ['1','2','3','4','5','6','7','8']
)

In [27]:
bodova_tabulka

# print(bodova_tabulka.to_latex())

,T [°C],p [MPa],i [kJ/kg],v [m3/kg],x [-]
1,370.000000,3.10,3160.749134,0.090933,1.0
2,332.731060,3.33,3066.801952,0.078112,1.0
3,348.000000,3.33,3103.887515,0.080685,1.0
4,240.564637,3.38,2803.002165,0.059114,1.0
5,240.564637,3.38,1040.218537,0.001231,0.0
6,236.000000,3.68,1018.575269,0.001220,0.0
7,217.000000,3.74,930.261487,0.001183,0.0
8,71.000000,4.00,300.442788,0.001022,0.0
